In [39]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
import feedparser
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import pdb

import numpy as np

In [39]:
FEED_URL='http://www.indeed.com/jobs?q=data+scientist&l=New+York%2C+NY' 
fp = feedparser.parse(FEED_URL)


In [109]:
fp = feedparser.parse(FEED_URL)
soup = BeautifulSoup(fp['feed']['summary'])

In [149]:
#method accepts a BeautifulSoup parsed HTML object and returns a list of <a> tags of job posts
def get_job_posts(soup_object):
    
    job_posts = []
    a_s = soup_object.find_all('a')
    for a in a_s:
        try:
            if u'jobtitle' in a['class']:
                job_posts.append(a)
        except:
            pass
    
    return job_posts

In [150]:
#method accepts a link to a job post and returns a list of 
def get_words(job_post_href):
    
    #only keep the content
    post_soup = BeautifulSoup(feedparser.parse(job_post_href)['feed']['summary'], 'html.parser')

    text = post_soup.get_text().lower()
    
    #keep only letters, but keep '+' for 'c++', '#' for 'c#','3' for 'd3, '!' for 'go!', and  '-' for 'scikit-learn'
    text = re.sub("[^a-z+#3-]"," ", text) 

    #break into lines to get rid of the annoying '\n' characters. Also project to lowercase.
    lines = [line.strip().lower() for line in text.splitlines()]

    words = []
    for line in lines:
        words += [each_word for each_word in line.split()]
        
    #Lighten the load by getting rid of basic stopwords, like "the", "or" etc    
    words = set(words) - set(nltk.corpus.stopwords.words("english")) 
    
    return words
    

In [173]:
#method accepts a list of words in a job post, and creates sets according to pre-determined sets
def words_by_category(job_posts):
    
    high_level_languages = set(['r', 'python', 'java', 'scala', 'c', 'c++', 'c#', 'c--', 
                               'f', 'f#', 'go', 'go!', 'groovy', 'julia', 'jscript', 
                               'matlab', 'perl'])
    
    low_level_languages = set(['pascal', 'haskell', 'fortran'])
    
    frameworks = set(['angular', 'angularjs', 'asp', 'node', 'django', 'flask' ])
    
    #operating systems and tools
    operating_systems = set(['unix', 'osx', 'bash', 'batch' , 'curl', 'linux' ])
    
    misc_tools = set(['latex' ])
    
    
    for post in job_posts:
        post['high_level_languages'] = set(post['words']) & high_level_languages
        post['low_level_languages'] = set(post['words']) & low_level_languages
        

In [160]:
def crawl_indeed(query, city = "", experience_level = "", num_pages = 10):
    
    #clean parameters so that they match Indeed's protocol
    
    #spaces parameters seperated by '+'
    query = query.replace(' ', '+')
    city = city.replace(' ' , '+')
    
    #make sure experience_level is one of the four valid options
    if experience_level not in ['', 'entry_level', 'mid_level', 'senior_level']:
        experience_level = ''
        print "Experience level parameter not valid. Showing all experience levels"
    
    #build data in JSON-like format, given the heirarchial nature of the data
    data = list()
    
    #Indeed shows job posts 10 at a time, so each page starts with post 0, 10, 20...
    #Loop through the posts using the num_pages parameter
    
    page_start_numbers = np.arange(num_pages)*10
    
    for start_number in page_start_numbers:
        
        url = "http://www.indeed.com/jobs?q={0}&l={1}&explvl={2}&start={3}".format(query, city, experience_level, start_number)
    
        #isolate html body and get rid of extraneous HTML objects using feedparser, and create a BeautifulSoup obect
        fp = feedparser.parse(url)
        page_soup = BeautifulSoup(fp['feed']['summary'])
    
        #get a list of all the <a class='jobtitle'></a> elemdents, which are job posts
        job_posts = get_job_posts(page_soup)

        for post in job_posts:
            
            #each job post gets a sub-dictionary
            post_data = dict()
            
            #attributes which are constant amongst all results
            post_data['query'] = query
            post_data['city'] = city
            
            #job title of that sepcific post
            post_data['title'] = post['title']
            
            
            #for each job post, extract the link to the post itself
            post_href = post['href']
            
            #extract all the (cleaned up) words fromt that link
            post_words = get_words(post_href)
                        
            post_data['words'] =post_words
            data.append(post_data)
            
    words_by_category(data)
    return data
            
            
    #http://www.indeed.com/jobs?q=data+scientist&l=New+York,+NY&explvl=entry_level
    #http://www.indeed.com/jobs?q=data+scientist&l=New+York,+NY&explvl=senior_level&start=0
    

In [174]:
data2 = crawl_indeed('data scientist', 'new york', num_pages = 2)

In [172]:
print a , b

a & b

set([1, 2, 4, 6]) set([2, 3, 4, 5, 6])


{2, 4, 6}

In [175]:
data2

[{'city': 'new+york',
  'high_level_languages': {'r'},
  'low_level_languages': set(),
  'query': 'data+scientist',
  'title': u'Data Scientist',
  'words': {u'+',
   u'-',
   u'3+',
   u'ability',
   u'able',
   u'aboutzestimatesresearchjobshelpadvertiseterms',
   u'access',
   u'accurate',
   u'advertising',
   u'advice',
   u'affordability',
   u'agent',
   u'agents',
   u'analysis',
   u'analytical',
   u'another',
   u'answer',
   u'apartments',
   u'app',
   u'apply',
   u'apps',
   u'area',
   u'around',
   u'atmosphere',
   u'automation',
   u'ba',
   u'back',
   u'bathroom',
   u'bathrooms',
   u'bedrooms',
   u'blog',
   u'blogs',
   u'boost',
   u'breadth',
   u'breaks',
   u'build',
   u'buildings',
   u'buy',
   u'buyers',
   u'buying',
   u'cabinets',
   u'calculator',
   u'calculators',
   u'candidate',
   u'career',
   u'careers',
   u'center',
   u'chances',
   u'chandelier',
   u'choiceblogmobile',
   u'city',
   u'collaborative',
   u'coming',
   u'communications',
 

In [84]:
a = set([1,4,2,6])
b = set([3,5,4,2,6]

In [92]:
a and b

{2, 3, 4, 5, 6}

In [13]:
job_post_href = 'https://careers.bloomberg.com/job/detail/54015?utm_source=Indeed&utm_campaign=Bloomberg_Indeed&sponsored=ppc&utm_campaign=SalesUS'
feedparser.parse(job_post_href)

post_soup = BeautifulSoup(feedparser.parse(job_post_href)['feed']['summary'], 'html.parser')
